<a href="https://colab.research.google.com/github/VavRe/information-retrieval-ut/blob/main/CA6/IR_CA6_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import zeros, int8, log
from pylab import random
import sys
import jieba
import re
import time
import codecs
from nltk.corpus import stopwords
import nltk
import pandas as pd
nltk.download('stopwords')
# segmentation, stopwords filtering and document-word matrix generating
# [return]:
# N : number of documents
# M : length of dictionary
# word2id : a map mapping terms to their corresponding ids
# id2word : a map mapping ids to terms
# X : document-word matrix, N*M, each line is the number of terms that show up in the document
def preprocessing(datasetFilePath):

    df=pd.read_csv('/content/drive/MyDrive/IR/BBC News.csv')
    documents=df['News'].values.tolist()[0:50]

    stops = set(stopwords.words("english"))
    # number of documents
    N = len(documents)

    wordCounts = [];
    word2id = {}
    id2word = {}
    currentId = 0;
    # generate the word2id and id2word maps and count the number of times of words showing up in documents
    for document in documents:
        segList = jieba.cut(document)
        wordCount = {}
        for word in segList:
            word = word.lower().strip()
            if len(word) > 1 and not re.search('[0-9]', word) and word not in stops:               
                if word not in word2id.keys():
                    word2id[word] = currentId;
                    id2word[currentId] = word;
                    currentId += 1;
                if word in wordCount:
                    wordCount[word] += 1
                else:
                    wordCount[word] = 1
        wordCounts.append(wordCount);
    
    # length of dictionary
    M = len(word2id)  

    # generate the document-word matrix
    X = zeros([N, M], int8)
    for word in word2id.keys():
        j = word2id[word]
        for i in range(0, N):
            if word in wordCounts[i]:
                X[i, j] = wordCounts[i][word];   
    #############
    ## My Work ##
    ############# 
    
    bg_model = zeros([M])
    corpus_words = sum(sum(X))
    for i in range(0,M):
        bg_model[i] = sum(X.T[i]) / corpus_words

    return N, M, word2id, id2word, X, bg_model

def initializeParameters():
    for i in range(0, N):
        normalization = sum(document_topic_dist[i, :])
        for j in range(0, K):
            document_topic_dist[i, j] /= normalization;

    for i in range(0, K):
        normalization = sum(topic_word_dist[i, :])
        for j in range(0, M):
            topic_word_dist[i, j] /= normalization;

def EStep(bg_model,lamda_b):
    for i in range(0, N):
        # print("E",i)
        for j in range(0, M):
            denominator = 0;
            for k in range(0, K):
                p[i, j, k] = topic_word_dist[k, j] * document_topic_dist[i, k];
                denominator += p[i, j, k];
            if denominator == 0:
                for k in range(0, K):
                    p[i, j, k] = 0;
            else:
                for k in range(0, K):
                    p[i, j, k] /= denominator;
            #############
            ## My Work ##
            ############# 
            denominator_bg = lamda_b * bg_model[j] + (1-lamda_b) * denominator
            p_bg[i,j,0] = (lamda_b * bg_model[j] ) / denominator_bg

def MStep():
    # update topic_word_dist
    for k in range(0, K):
        denominator = 0
        for j in range(0, M):
            topic_word_dist[k, j] = 0
            for i in range(0, N):
                #############
                ## My Work ##
                ############# 
                topic_word_dist[k, j] += X[i, j] * p[i, j, k] * (1 - p_bg[i,j,0] )
            denominator += topic_word_dist[k, j]
        if denominator == 0:
            for j in range(0, M):
                topic_word_dist[k, j] = 1.0 / M
        else:
            for j in range(0, M):
                topic_word_dist[k, j] /= denominator
        
    # update document_topic_dist
    for i in range(0, N):
        for k in range(0, K):
            document_topic_dist[i, k] = 0
            denominator = 0
            for j in range(0, M):
                #############
                ## My Work ##
                #############
                document_topic_dist[i, k] += X[i, j] * p[i, j, k] * (1 - p_bg[i,j,0] )
                denominator += X[i, j];
            if denominator == 0:
                document_topic_dist[i, k] = 1.0 / K
            else:
                document_topic_dist[i, k] /= denominator

# calculate the log likelihood
def LogLikelihood(bg_model,lamda_b):
    loglikelihood = 0
    for i in range(0, N):
        for j in range(0, M):
            tmp = 0
            for k in range(0, K):
                tmp += topic_word_dist[k, j] * document_topic_dist[i, k]
            #############
            ## My Work ##
            #############
            tmp = lamda_b * bg_model[j]+(1-lamda_b)*tmp
            if tmp > 0:
                loglikelihood += X[i, j] * log(tmp)
    return loglikelihood

# output the params of model and top words of topics to files
def output():
    # document-topic distribution
    file = codecs.open(docTopicDist,'w','utf-8')
    for i in range(0, N):
        tmp = ''
        for j in range(0, K):
            tmp += str(document_topic_dist[i, j]) + ' '
        file.write(tmp + '\n')
    file.close()
    
    # topic-word distribution
    file = codecs.open(topicWordDist,'w','utf-8')
    for i in range(0, K):
        tmp = ''
        for j in range(0, M):
            tmp += str(topic_word_dist[i, j]) + ' '
        file.write(tmp + '\n')
    file.close()
    
    # dictionary
    file = codecs.open(dictionary,'w','utf-8')
    for i in range(0, M):
        file.write(id2word[i] + '\n')
    file.close()
    
    # top words of each topic
    file = codecs.open(topicWords,'w','utf-8')
    for i in range(0, K):
        topicword = []
        ids = topic_word_dist[i, :].argsort()
        for j in ids:
            topicword.insert(0, id2word[j])
        tmp = ''
        for word in topicword[0:min(topicWordsNum, len(topicword))]:
            tmp += word + ' '
        file.write(tmp + '\n')
    file.close()
    
# set the default params and read the params from cmd
datasetFilePath = 'dataset.txt'
# K = 20    # choose different number of topics
maxIteration = 10
threshold = 0.0001
topicWordsNum = 10
dictionary = 'dictionary.dic'
docTopicDist = f'docTopicDistribution-k={K}-lamda-{lamda_b}.txt'
topicWordDist = f'topicWordDistribution-k={K}-lamda={lamda_b}.txt'
topicWords = f'topics-k={K}-lamda={lamda_b}.txt'
lamda_b = 0.3


# preprocessing
N, M, word2id, id2word, X , bg_model = preprocessing(datasetFilePath)

# # document_topic_dist[i, j] : p(zj|di)
# document_topic_dist = random([N, K])

# # topic_word_dist[i, j] : p(wj|zi)
# topic_word_dist = random([K, M])

# # p[i, j, k] : p(zk|di,wj)
# p = zeros([N, M, K])
# p_bg = zeros([N,M,1])

# initializeParameters()

# # EM algorithm
# oldLoglikelihood = 1
# newLoglikelihood = 1
# for i in range(0, maxIteration):
#     EStep(bg_model,lamda_b)
#     MStep()
#     newLoglikelihood = LogLikelihood(bg_model,lamda_b)
#     print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
#     if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
#         break
#     oldLoglikelihood = newLoglikelihood

# output()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.182 seconds.
DEBUG:jieba:Loading model cost 2.182 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
topic_counts = [5,10,20,30,50]
lamdas = [0,0.4,0.8,1]
for K in topic_counts:
  for lamda_b in lamdas:
    docTopicDist = f'/content/drive/MyDrive/IR/results/docTopicDistribution-k={K}-lamda-{lamda_b}.txt'
    topicWordDist = f'/content/drive/MyDrive/IR/results/topicWordDistribution-k={K}-lamda={lamda_b}.txt'
    topicWords = f'/content/drive/MyDrive/IR/results/topics-k={K}-lamda={lamda_b}.txt'
    print(f"k is {K} and lamda is {lamda_b}")
    # document_topic_dist[i, j] : p(zj|di)
    document_topic_dist = random([N, K])

    # topic_word_dist[i, j] : p(wj|zi)
    topic_word_dist = random([K, M])

    # p[i, j, k] : p(zk|di,wj)
    p = zeros([N, M, K])
    p_bg = zeros([N,M,1])

    initializeParameters()

    # EM algorithm
    oldLoglikelihood = 1
    newLoglikelihood = 1
    for i in range(0, maxIteration):
        EStep(bg_model,lamda_b)
        MStep()
        newLoglikelihood = LogLikelihood(bg_model,lamda_b)
        print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
        if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
            break
        oldLoglikelihood = newLoglikelihood

    output()


k is 5 and lamda is 0
[ 2023-01-20 17:05:19 ]  1  iteration   -77620.66565187421
[ 2023-01-20 17:05:29 ]  2  iteration   -76649.19354944075
[ 2023-01-20 17:05:39 ]  3  iteration   -75539.78388459676
[ 2023-01-20 17:05:49 ]  4  iteration   -74382.67862632895
[ 2023-01-20 17:06:00 ]  5  iteration   -73288.62267050912
[ 2023-01-20 17:06:10 ]  6  iteration   -72321.62350657741
[ 2023-01-20 17:06:20 ]  7  iteration   -71546.27531316585
[ 2023-01-20 17:06:30 ]  8  iteration   -70988.74966550535
[ 2023-01-20 17:06:40 ]  9  iteration   -70610.68348530716
[ 2023-01-20 17:06:50 ]  10  iteration   -70353.41016259546
k is 5 and lamda is 0.4
[ 2023-01-20 17:07:00 ]  1  iteration   -82335.68408022719
[ 2023-01-20 17:07:12 ]  2  iteration   -82173.06229217949
[ 2023-01-20 17:07:22 ]  3  iteration   -81218.25538631281
[ 2023-01-20 17:07:32 ]  4  iteration   -79866.67212241674
[ 2023-01-20 17:07:42 ]  5  iteration   -78544.37968169138
[ 2023-01-20 17:07:52 ]  6  iteration   -77538.89349976394
[ 2023-01

To Calculate lamda effect on LogLikelihood

In [ ]:
topic_counts = [5]
lamdas = [0,0.3,0.4,0.5,0.7,0.8,1]
for K in topic_counts:
  for lamda_b in lamdas:
    docTopicDist = f'/content/drive/MyDrive/IR/results/docTopicDistribution-k={K}-lamda-{lamda_b}.txt'
    topicWordDist = f'/content/drive/MyDrive/IR/results/topicWordDistribution-k={K}-lamda={lamda_b}.txt'
    topicWords = f'/content/drive/MyDrive/IR/results/topics-k={K}-lamda={lamda_b}.txt'
    print(f"k is {K} and lamda is {lamda_b}")
    # document_topic_dist[i, j] : p(zj|di)
    document_topic_dist = random([N, K])

    # topic_word_dist[i, j] : p(wj|zi)
    topic_word_dist = random([K, M])

    # p[i, j, k] : p(zk|di,wj)
    p = zeros([N, M, K])
    p_bg = zeros([N,M,1])

    initializeParameters()

    # EM algorithm
    oldLoglikelihood = 1
    newLoglikelihood = 1
    for i in range(0, maxIteration):
        EStep(bg_model,lamda_b)
        MStep()
        newLoglikelihood = LogLikelihood(bg_model,lamda_b)
        print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
        if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
            break
        oldLoglikelihood = newLoglikelihood

    output()


k is 5 and lamda is 0
[ 2023-01-20 19:41:14 ]  1  iteration   -77775.05699560874
[ 2023-01-20 19:41:24 ]  2  iteration   -76801.29270465448
[ 2023-01-20 19:41:35 ]  3  iteration   -75591.45614928237
[ 2023-01-20 19:41:45 ]  4  iteration   -74298.23885814568
[ 2023-01-20 19:41:55 ]  5  iteration   -73131.81939371089
[ 2023-01-20 19:42:06 ]  6  iteration   -72178.1196733101
[ 2023-01-20 19:42:18 ]  7  iteration   -71420.58818584416
[ 2023-01-20 19:42:28 ]  8  iteration   -70848.59718684759
[ 2023-01-20 19:42:39 ]  9  iteration   -70464.13215534034
[ 2023-01-20 19:42:49 ]  10  iteration   -70219.93080239034
k is 5 and lamda is 0.3
[ 2023-01-20 19:42:59 ]  1  iteration   -82349.959986092
[ 2023-01-20 19:43:10 ]  2  iteration   -81441.92216079264
[ 2023-01-20 19:43:20 ]  3  iteration   -79958.56023322252
[ 2023-01-20 19:43:30 ]  4  iteration   -78301.97740443311
[ 2023-01-20 19:43:40 ]  5  iteration   -76804.63201446735
[ 2023-01-20 19:43:51 ]  6  iteration   -75612.69950110362
[ 2023-01-20